<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/mymap_image_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

In [ ]:
import requests
import pandas as pd
from lxml import etree

In [ ]:
import time

In [ ]:
import imghdr
import io
import pathlib

# 設定

In [ ]:
# マイマップのmid
mid = "1j-z0M8cmRv3qcFgsHQYnY_hYHQRi58AZ"
lid = ""

# マイマップからKMZをダウンロード

In [ ]:
url = f"https://www.google.com/maps/d/kml?mid={mid}" if lid == "" else f"https://www.google.com/maps/d/kml?mid={mid}&lid={lid}"
url

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko"
}

In [ ]:
!wget "$url" -O rakuten.kmz

# KMZからCSV変換

In [ ]:
with zipfile.ZipFile("rakuten.kmz") as existing_zip:
    existing_zip.extractall("kmz")

In [ ]:
ns = {"kml": "http://www.opengis.net/kml/2.2"}
tree = etree.parse("./kmz/doc.kml")

In [ ]:
data = []

for i in tree.xpath("//kml:Placemark", namespaces=ns):

    d = {}

    d["場所"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
    d["geometry"] = "".join(
        i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
    ).strip()

    for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

        k = j.attrib["name"].strip()
        v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

        d[k] = v

    data.append(d)

In [ ]:
df0 = pd.DataFrame(data)

In [ ]:
df0.shape

In [ ]:
df0[["経度", "緯度", "標高"]] = (
    df0["geometry"]
    .str.split(",", expand=True)
)

In [ ]:
# 文字から数値に変換
df0["経度"] = pd.to_numeric(df0["経度"])
df0["緯度"] = pd.to_numeric(df0["緯度"])

In [ ]:
df0

In [ ]:
df0["gx_media_links"].isnull().sum()

In [ ]:
df1 = df0.dropna(subset=["gx_media_links"]).dropna(subset=["緯度", "経度"], how="any").copy()

In [ ]:
df1.to_csv("list.csv", encoding="utf_8_sig")

In [ ]:
df2 = df1.loc[:, ["場所", "gx_media_links"]]

In [ ]:
df3 = df2["gx_media_links"].str.split(expand=True)

In [ ]:
df3.columns = df3.columns.map(lambda x: x+1)

In [ ]:
df3

In [ ]:
df4 = df2.join(df3)

In [ ]:
df4

In [ ]:
df4.drop("gx_media_links", axis=1, inplace=True)

In [ ]:
df5 = df4.melt(id_vars="場所").dropna(subset=["value"])

In [ ]:
df5

In [ ]:
df5["value"] = df5["value"].str.replace("fife", "fife=s16383")

In [ ]:
df5["場所"] = df5["場所"].str.replace("【.+】", "", regex=True).str.normalize("NFKC")

In [ ]:
def fetch_file(url, filename, dir="."):

    r = requests.get(url, headers=headers)
    r.raise_for_status()

    # 画像種類判定
    suffix = imghdr.what(io.BytesIO(r.content))

    p = pathlib.Path(dir, filename)
    p.parent.mkdir(parents=True, exist_ok=True)

    # 拡張子指定
    p_suffix =  p.with_suffix(f".{suffix}")

    with p_suffix.open(mode="wb") as fw:
        fw.write(r.content)
        
    return p_suffix

In [ ]:
for i, row in df5.iterrows():

    fetch_file(row["value"], f'{row["場所"]}_{row["variable"]:02}', "download")

    time.sleep(2)

In [ ]:
!zip -r download.zip download

In [ ]:
from google.colab import files

files.download("download.zip")